In [7]:
import pandas as pd
from pprint import pprint
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import copy
import matplotlib.pyplot as plt
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, mean_squared_error
from sklearn.metrics import r2_score as sklearn_r2_score
import time
from sklearn.linear_model import LinearRegression
import numpy as np

In [8]:


dataset = pd.read_csv("housing.csv")
# dataset.head()
# dataset = dataset.fillna(dataset.median())
X = dataset.drop(columns=["median_house_value", 'ocean_proximity'])
y = dataset['median_house_value']



# pd.get_dummies(dataset['ocean_proximity'])

M, N = X.shape
X['total_bedrooms'] = X['total_bedrooms'].fillna(X['total_bedrooms'].median())


columns = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income']
scaler = StandardScaler()
X[columns] =  scaler.fit_transform(X[columns])
y = StandardScaler().fit_transform(dataset[['median_house_value']]).reshape((-1))

X = pd.concat([X, pd.get_dummies(dataset['ocean_proximity'])], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# print(X.isna().sum())
X.head()
X_train = X_train.to_numpy()

X_test = X_test.to_numpy()

print(y)

[ 2.12963148  1.31415614  1.25869341 ... -0.99274649 -1.05860847
 -1.01787803]


In [9]:
model = LinearRegression()

start = time.time()
reg = model.fit(X_train, y_train)
time_taken = time.time() - start

y_train_prediction = reg.predict(X_train)
y_test_prediction = reg.predict(X_test)


print("Time taken: ", time_taken)
print()

print("On training dataset:")
print("MSE - ", mean_squared_error(y_train, y_train_prediction)/2)
print("RMSE - ", root_mean_squared_error(y_train, y_train_prediction)/2**0.5)
print("MAE - ", mean_absolute_error(y_train, y_train_prediction))
print("R2-Score - ", sklearn_r2_score(y_train, y_train_prediction))
print()

print("On test dataset:")
print("MSE - ", mean_squared_error(y_test, y_test_prediction)/2)
print("RMSE - ", root_mean_squared_error(y_test, y_test_prediction)/2**0.5)
print("MAE - ", mean_absolute_error(y_test, y_test_prediction))
print("R2-Score - ", sklearn_r2_score(y_test, y_test_prediction))



Time taken:  0.016056537628173828

On training dataset:
MSE -  0.17755556968690311
RMSE -  0.42137343258314597
MAE -  0.4309052334205368
R2-Score -  0.6470480227253683

On test dataset:
MSE -  0.17774504173074568
RMSE -  0.4215981993922005
MAE -  0.43397932961673547
R2-Score -  0.6393611711434393


In [10]:
X_new = np.concatenate([np.ones((X_train.shape[0], 1)), X_train], axis=1).astype(np.float64)




params = np.linalg.inv(X_new.T@X_new)@X_new.T@y_train

In [11]:
print(params)

print(reg.intercept_,reg.coef_)

[-0.62704426 -0.3144845  -0.24355345  0.12667885  0.00192418  0.23981207
 -0.51827097  0.33612177  0.59744891  0.48252612 -0.37951134  1.00803701
 -0.32400445  0.23312738]
0.2729236293506678 [-0.46013883 -0.46118261  0.12062252 -0.11101941  0.3842319  -0.36631703
  0.1401925   0.64651846 -0.15931572 -0.51528292  1.01830127 -0.21188439
 -0.13181823]


In [13]:
W = params[1:]
b = params[0]

mse = mean_squared_error(y_test, X_test@W+b)/2
print(mse)

0.3536513023032572


The above error shows that using the formula fails because of linearly dependent columns. lets try again now by removing 1 column from one hot encodings

In [30]:
X_train_new = np.delete(X_train, 12, 1).astype(np.float64)

X_new = np.concatenate([np.ones((X_train_new.shape[0], 1)), X_train_new], axis=1).astype(np.float64)


X_new.T@X_new
np.linalg.inv(X_new.T@X_new)
params = np.linalg.inv(X_new.T@X_new)@X_new.T@y_train

W = params[1:]
b = params[0]
print(W)
print(b)
mse = mean_squared_error(y_train, X_train_new@W+b)/2
print(mse)

[-0.46013883 -0.46118261  0.12062252 -0.11101941  0.3842319  -0.36631703
  0.1401925   0.64651846 -0.02749749 -0.38346469  1.1501195  -0.08006616]
0.1411053971162745
0.1775555696869031


In [34]:
reg = LinearRegression().fit(X_train_new, y_train)
print(reg.coef_)
print(reg.intercept_)
mse = mean_squared_error(y_train, reg.predict(X_train_new))/2
print(mse)



[-0.46013883 -0.46118261  0.12062252 -0.11101941  0.3842319  -0.36631703
  0.1401925   0.64651846 -0.02749749 -0.38346469  1.1501195  -0.08006616]
0.14110539711627756
0.17755556968690311


As we can see, on dropping a column from one hot encoding, and doing the above method, we get exactly the same  answer as scikit-learn